# One shot object condensation

This notebook shows how you can implement a model that directly goes from point cloud data to object condensation.

In [1]:
from pytorch_lightning.core.mixins import HyperparametersMixin
from torch import nn
from torch_geometric.nn.conv import GravNetConv
from torch_geometric.data import Data
from pytorch_lightning import Trainer

from gnn_tracking.metrics.losses import PotentialLoss, BackgroundLoss
import torch
from functools import partial

from gnn_tracking.training.callbacks import PrintValidationMetrics
from gnn_tracking.training.tc import TCModule
from gnn_tracking.utils.loading import TrackingDataModule

## 1. Configure data

In [2]:
dm = TrackingDataModule(
    train=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        stop=5,
    ),
    val=dict(
        dirs=["/Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/"],
        start=5,
        stop=10,
    ),
    # could also configure a 'test' set here
)

## 2. Write a model

In [3]:
class DemoGravNet(nn.Module, HyperparametersMixin):
    def __init__(self, in_dim: int = 14, depth: int = 1, k: int = 2):
        super().__init__()
        self.save_hyperparameters()
        layers = [
            GravNetConv(
                in_channels=in_dim,
                out_channels=in_dim,
                space_dimensions=3,
                propagate_dimensions=3,
                k=k,
            )
            for _ in range(depth)
        ]
        self._embedding = nn.Sequential(*layers)
        self._beta = nn.Sequential(
            nn.Linear(in_dim, 1),
            nn.Sigmoid(),
        )

    def forward(self, data: Data):
        latent = self._embedding(data.x)
        beta = self._beta(latent).squeeze()
        eps = 1e-6
        beta = beta.clamp(eps, 1 - eps)
        return {
            "B": beta,
            "H": latent,
        }

In [4]:
model = DemoGravNet()

## 3. Configure loss functions and weights

In [10]:
# The loss functions can be memory hungry. Here we override `data_preproc` to place a tighter pt cut on
# the data to easy computation (since this is just a demo).
class PtCut(HyperparametersMixin):
    def __call__(self, data: Data):
        mask = data.pt > 4
        data = data.subgraph(mask)
        return data

In [12]:
from gnn_tracking.postprocessing.dbscanscanner import DBSCANHyperParamScanner

# TC for track condensation
lmodel = TCModule(
    model=model,
    potential_loss=PotentialLoss(
        radius_threshold=1.0,
    ),
    background_loss=BackgroundLoss(),
    lw_repulsive=2.0,
    lw_background=0.1,
    optimizer=partial(torch.optim.Adam, lr=1e-4),
    cluster_scanner=DBSCANHyperParamScanner(n_trials=5, n_jobs=1),
    preproc=PtCut(),
)

[15:06:34] DEBUG: Got obj of type <class '__main__.DemoGravNet'>, assuming I have to save hyperparameters
[15:06:34] DEBUG: Saving hyperperameters {'class_path': '__main__.DemoGravNet', 'init_args': {'in_dim': 14, 'depth': 1, 'k': 2}}
[15:06:34] DEBUG: Got obj of type <class '__main__.PtCut'>, assuming I have to save hyperparameters
[15:06:34] DEBUG: Saving hyperperameters {'class_path': '__main__.PtCut', 'init_args': {}}
[15:06:34] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.PotentialLoss'>, assuming I have to save hyperparameters
[15:06:34] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.PotentialLoss', 'init_args': {'q_min': 0.01, 'radius_threshold': 1.0, 'attr_pt_thld': 0.9}}
[15:06:34] DEBUG: Got obj of type <class 'gnn_tracking.metrics.losses.BackgroundLoss'>, assuming I have to save hyperparameters
[15:06:34] DEBUG: Saving hyperperameters {'class_path': 'gnn_tracking.metrics.losses.BackgroundLoss', 'init_args': {'sb': 0.1}}
[15:06:34] DEB

## 4. Train the model

In [13]:
trainer = Trainer(
    max_epochs=1,
    accelerator="cpu",
    log_every_n_steps=1,
    callbacks=[PrintValidationMetrics()],
)
trainer.fit(model=lmodel, datamodule=dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
[15:06:36] INFO: DataLoader will load 5 graphs (out of 90 available).
[15:06:36] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21025_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21053_s0.pt
[15:06:36] INFO: DataLoader will load 5 graphs (out of 90 available).
[15:06:36] DEBUG: First graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21058_s0.pt, last graph is /Users/fuchur/tmp/truth_cut_graphs_for_gsoc/part_1_0/data21094_s0.pt

  | Name            | Type           | Params
------------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


                   Validation epoch=0                   
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric              ┃          Value ┃         Error ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ attractive          │ 13405383.00000 │ 6558321.50000 │
│ attractive_weighted │ 13405383.00000 │ 6558321.50000 │
│ background          │        0.65469 │       0.11550 │
│ background_weighted │        0.06547 │       0.01155 │
│ repulsive           │        0.03030 │       0.03342 │
│ repulsive_weighted  │        0.06060 │       0.06684 │
│ total               │ 13405383.00000 │ 6558321.50000 │
└─────────────────────┴────────────────┴───────────────┘

/Users/fuchur/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[15:06:51 ClusterHP] DEBUG: Starting from params: {}
[15:06:51 ClusterHP] INFO: Starting hyperparameter scan for clustering
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return a / b
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return a / b
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return a / b
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return a / b
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeWarning: invalid value encountered in scalar divide
  return a / b
/Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking/utils/math.py:4: RuntimeW

                        Validation epoch=0                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric                         ┃          Value ┃         Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ adjusted_rand                  │        0.02026 │       0.01163 │
│ attractive                     │ 21853480.00000 │ 7339746.00000 │
│ attractive_weighted            │ 21853480.00000 │ 7339746.00000 │
│ background                     │        0.58068 │       0.06895 │
│ background_weighted            │        0.05807 │       0.00690 │
│ best_dbscan_eps                │        0.70959 │           nan │
│ best_dbscan_min_samples        │        1.00000 │           nan │
│ completeness                   │        0.64601 │       0.01496 │
│ fowlkes_mallows                │        0.06771 │       0.03675 │
│ homogeneity                    │        0.98101 │       0.01501 │
│ repulsive                      │        4.00483 │       2.85446 │
│ repulsive_weighted             │        8.00967 │       5.70892 │
│ total                          │ 21853488.00000 │ 7339746.00000 │
│ trk.double_majority            │        0.00893 │       0.01525 │
│ trk.double_majority_pt0.5      │        0.00893 │       0.01525 │
│ trk.double_majority_pt0.9      │        0.00893 │       0.01525 │
│ trk.double_majority_pt1.5      │        0.00893 │       0.01525 │
│ trk.fake_double_majority       │        0.01530 │       0.01938 │
│ trk.fake_double_majority_pt0.5 │        0.01530 │       0.01938 │
│ trk.fake_double_majority_pt0.9 │        0.01530 │       0.01938 │
│ trk.fake_double_majority_pt1.5 │        0.01530 │       0.01938 │
│ trk.fake_lhc                   │        0.60000 │       0.54772 │
│ trk.fake_lhc_pt0.5             │        0.60000 │       0.54772 │
│ trk.fake_lhc_pt0.9             │        0.60000 │       0.54772 │
│ trk.fake_lhc_pt1.5             │        0.60000 │       0.54772 │
│ trk.fake_perfect               │        0.02422 │       0.01702 │
│ trk.fake_perfect_pt0.5         │        0.02422 │       0.01702 │
│ trk.fake_perfect_pt0.9         │        0.02422 │       0.01702 │
│ trk.fake_perfect_pt1.5         │        0.02422 │       0.01702 │
│ trk.lhc                        │        0.40000 │       0.54772 │
│ trk.lhc_pt0.5                  │        0.40000 │       0.54772 │
│ trk.lhc_pt0.9                  │        0.40000 │       0.54772 │
│ trk.lhc_pt1.5                  │        0.40000 │       0.54772 │
│ trk.n_cleaned_clusters         │        0.85714 │       0.69007 │
│ trk.n_cleaned_clusters_pt0.5   │        0.85714 │       0.69007 │
│ trk.n_cleaned_clusters_pt0.9   │        0.85714 │       0.69007 │
│ trk.n_cleaned_clusters_pt1.5   │        0.85714 │       0.69007 │
│ trk.n_particles                │       32.42857 │       7.89213 │
│ trk.n_particles_pt0.5          │       32.42857 │       7.89213 │
│ trk.n_particles_pt0.9          │       32.42857 │       7.89213 │
│ trk.n_particles_pt1.5          │       32.42857 │       7.89213 │
│ trk.perfect                    │        0.00000 │       0.00000 │
│ trk.perfect_pt0.5              │        0.00000 │       0.00000 │
│ trk.perfect_pt0.9              │        0.00000 │       0.00000 │
│ trk.perfect_pt1.5              │        0.00000 │       0.00000 │
│ v_measure                      │        0.77888 │       0.01152 │
└────────────────────────────────┴────────────────┴───────────────┘

`Trainer.fit` stopped: `max_epochs=1` reached.
